In [1]:
#Data Preparation
#Here's an example of loading and preprocessing the datase
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

data_path = 'Downloads/sign_lang'
labels = {'hello': 0, 'thanks': 1, 'please': 2}  # Example labels
images = []
targets = []

for label in labels:
    sign_path = os.path.join(data_path, label)
    for image_file in os.listdir(sign_path):
        img_path = os.path.join(sign_path, image_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        images.append(img)
        targets.append(labels[label])

images = np.array(images) / 255.0
targets = np.array(targets)

X_train, X_val, y_train, y_val = train_test_split(images, targets, test_size=0.2, random_state=42)

In [2]:
#Model Creation
#Build a CNN for image classification:


import tensorflow as tf
from tensorflow.keras import layers, models

def create_sign_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(len(labels), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

sign_model = create_sign_model()


C:\Users\harrr\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
#Model Training
#Train the model:


sign_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
sign_model.save('sign_model.h5')

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 550ms/step - accuracy: 0.2708 - loss: 1.3790 - val_accuracy: 0.3636 - val_loss: 1.4885
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.4188 - loss: 1.2726 - val_accuracy: 0.2727 - val_loss: 1.1453
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.3583 - loss: 1.0876 - val_accuracy: 0.2727 - val_loss: 1.1046
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.5000 - loss: 1.0636 - val_accuracy: 0.2727 - val_loss: 1.1418
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.6188 - loss: 0.9921 - val_accuracy: 0.1818 - val_loss: 1.1564
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.7500 - loss: 0.9086 - val_accuracy: 0.1818 - val_loss: 1.2872
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.6729 - loss: 0.8675 - val_accuracy: 0.1818 - val_loss: 1.2953
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.7063 - loss: 0.7542 - val_accuracy: 0.3636 - val_loss:

In [8]:


#GUI Development with Time-based Operation
#Create a GUI using tkinter with features for image upload and real-time video processing, and include the time-based operational restriction:



In [7]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from datetime import datetime
import cv2
from PIL import Image, ImageTk

def within_operational_hours():
    current_time = datetime.now().time()
    return current_time >= datetime.strptime('18:00', '%H:%M').time() and current_time <= datetime.strptime('22:00', '%H:%M').time()

def check_time_and_operate(func):
    def wrapper(*args, **kwargs):
        if within_operational_hours():
            return func(*args, **kwargs)
        else:
            messagebox.showwarning("Operation Not Allowed", "Operations are only allowed between 6 PM and 10 PM")
    return wrapper

@check_time_and_operate
def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((128, 128), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        panel.config(image=img)
        panel.image = img
        predict_image(file_path)

@check_time_and_operate
def predict_image(file_path):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (128, 128)) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = sign_model.predict(img)
    predicted_label = np.argmax(prediction[0])
    label.config(text=f"Predicted Sign: {list(labels.keys())[predicted_label]}")

@check_time_and_operate
def predict_video():
    cap = cv2.VideoCapture(0)
    while within_operational_hours() and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.resize(frame, (128, 128)) / 255.0
        img = np.expand_dims(img, axis=0)
        prediction = sign_model.predict(img)
        predicted_label = np.argmax(prediction[0])
        cv2.putText(frame, f"Predicted: {list(labels.keys())[predicted_label]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("Sign Language Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Create the main window
root = tk.Tk()
root.title("Sign Language Recognition")

# Create and pack widgets
upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
upload_btn.pack(pady=20)

video_btn = tk.Button(root, text="Real-time Video", command=predict_video)
video_btn.pack(pady=20)

detect_btn = tk.Button(root, text="Detect Sign Language", command=check_time_and_operate(predict_video))
detect_btn.pack(pady=20)

panel = tk.Label(root)
panel.pack(pady=20)

label = tk.Label(root, text="")
label.pack(pady=20)

# Run the application
root.mainloop()

In [10]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import cv2
from PIL import Image, ImageTk

def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((128, 128), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        panel.config(image=img)
        panel.image = img
        predict_image(file_path)

def predict_image(file_path):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (128, 128)) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = sign_model.predict(img)
    predicted_label = np.argmax(prediction[0])
    label.config(text=f"Predicted Sign: {list(labels.keys())[predicted_label]}")

def predict_video():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.resize(frame, (128, 128)) / 255.0
        img = np.expand_dims(img, axis=0)
        prediction = sign_model.predict(img)
        predicted_label = np.argmax(prediction[0])
        cv2.putText(frame, f"Predicted: {list(labels.keys())[predicted_label]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("Sign Language Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Create the main window
root = tk.Tk()
root.title("Sign Language Recognition")

# Create and pack widgets
upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
upload_btn.pack(pady=20)

video_btn = tk.Button(root, text="Real-time Video", command=predict_video)
video_btn.pack(pady=20)

panel = tk.Label(root)
panel.pack(pady=20)

label = tk.Label(root, text="")
label.pack(pady=20)

# Run the application
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\harrr\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\harrr\AppData\Local\Temp\ipykernel_5316\3476143625.py", line 11, in upload_image
    img = img.resize((128, 128), Image.ANTIALIAS)
                                 ^^^^^^^^^^^^^^^
AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'
